In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
config = {
    'domain': 'protein',
    'postgres': False,
    'debug': False,
#     'max_docs': 3,
    'babbler_candidate_split': 0,
    'babbler_label_split': 2,
    'supervision': 'majority_vote',
    'do_filter_duplicate_semantics': False, 
    'do_filter_consistency': False, 
    'do_filter_duplicate_signatures': False, 
    'do_filter_uniform_signatures': False,
}

In [25]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

default_db_name = 'babble_' + config['domain'] + ('_debug' if config.get('debug', False) else '')
DB_NAME = config.get('db_name', default_db_name)
if 'postgres' in config and config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = sqlite:///babble_protein.db


In [26]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import merge_configs, get_local_pipeline
config = merge_configs(config)

if config['debug']:
    print("NOTE: --debug=True: modifying parameters...")
    config['max_docs'] = 100
    config['gen_model_search_space'] = 2
    config['disc_model_search_space'] = 2
    config['gen_params_default']['epochs'] = 25
    config['disc_params_default']['n_epochs'] = 5

from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

Overwriting supervision=generative to supervision=majority_vote


In [5]:
%time pipe.parse()

Clearing existing...
Running UDF...
[========================================] 100%

Documents: 53
Sentences: 1551
CPU times: user 3.75 s, sys: 142 ms, total: 3.89 s
Wall time: 49 s


In [6]:
%time pipe.extract()

Clearing existing...
Running UDF...
[========================================] 100%

Candidates [Split 0]: 664
Clearing existing...
Running UDF...
[========================================] 100%

Candidates [Split 1]: 126
Clearing existing...
Running UDF...
[========================================] 100%

Candidates [Split 2]: 203
CPU times: user 3.23 s, sys: 236 ms, total: 3.47 s
Wall time: 3.35 s


In [23]:
%time pipe.load_gold()

AnnotatorLabels created: 664
AnnotatorLabels created: 0
AnnotatorLabels created: 203
CPU times: user 14 s, sys: 202 ms, total: 14.2 s
Wall time: 14.3 s


In [8]:
%time pipe.collect()

Skipping malformed or header row 23...
Linking candidates...
Building list of target candidate ids...
Collected 29 unique target candidate ids from 56 explanations.
Gathering desired candidates...
Found 29/29 desired candidates
Linking explanations to candidates...
Linked 56/56 explanations
Calling babbler...
Created grammar with 480 rules
57 parses created from 41 out of 56 explanation(s)
Parsed 57 LFs from 56 explanations.
Filtered to 40 LFs with duplicate semantics filter (17 filtered).
Filtered to 23 LFs with consistency filter (17 filtered).
Clearing existing...
Running UDF...
[========================================] 100%

Filtered to 6 LFs with uniform signatures filter (17 filtered).
Filtered to 6 LFs with duplicate signatures filter (0 filtered).
CPU times: user 7.01 s, sys: 478 ms, total: 7.49 s
Wall time: 7.47 s


In [12]:
%time pipe.label()

Clearing existing...
Running UDF...
[========================================] 100%


Labeled split 0: (664,6) sparse (nnz = 104)
Clearing existing...
Running UDF...
[========================================] 100%


Labeled split 1: (126,6) sparse (nnz = 23)
                 j  Coverage  Overlaps  Conflicts  TP  FP  FN  TN  \
Explanation2_0   0  0.015873  0.015873   0.007937   0   2   0   0   
Explanation33_1  1  0.039683  0.000000   0.000000   0   0   0   5   
Explanation5_1   2  0.007937  0.000000   0.000000   0   0   0   1   
Explanation50_0  3  0.055556  0.023810   0.015873   0   7   0   0   
Explanation51_1  4  0.055556  0.023810   0.023810   0   0   1   6   
Explanation6_0   5  0.007937  0.000000   0.000000   0   0   0   1   

                 Empirical Acc.  
Explanation2_0         0.000000  
Explanation33_1        1.000000  
Explanation5_1         1.000000  
Explanation50_0        0.000000  
Explanation51_1        0.857143  
Explanation6_0         1.000000  
Clearing existing..

In [40]:
%time pipe.supervise()

Using L_train: <664x6 sparse matrix of type '<type 'numpy.int64'>'
	with 104 stored elements in Compressed Sparse Row format>
Using L_gold_train: <664x1 sparse matrix of type '<type 'numpy.int64'>'
	with 664 stored elements in Compressed Sparse Row format>
Positive Fraction: 1.5%

Using L_dev: <126x6 sparse matrix of type '<type 'numpy.int64'>'
	with 23 stored elements in Compressed Sparse Row format>
Using L_gold_dev: <126x1 sparse matrix of type '<type 'numpy.int64'>'
	with 126 stored elements in Compressed Sparse Row format>
Positive Fraction: 7.1%

Using L_test: <203x6 sparse matrix of type '<type 'numpy.int64'>'
	with 15 stored elements in Compressed Sparse Row format>
Using L_gold_test: <203x1 sparse matrix of type '<type 'numpy.int64'>'
	with 203 stored elements in Compressed Sparse Row format>
Positive Fraction: 5.9%

Saved 664 marginals
CPU times: user 42.2 ms, sys: 4.7 ms, total: 46.9 ms
Wall time: 45.5 ms


In [28]:
%time pipe.classify()

### [7.1] Begin training discriminative model
[1] Testing dim = 64, dropout = 2.50e-01, rebalance = 0.00e+00, lr = 1.00e-03


/Users/bradenjh/anaconda/envs/snorkel/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[reRNN] Training model
[reRNN] n_train=664  #epochs=20  batch size=128
[reRNN] Epoch 0 (1.57s)	Average loss=0.677659	Dev F1=0.00
[reRNN] Epoch 1 (3.19s)	Average loss=0.567539	Dev F1=0.00
[reRNN] Epoch 2 (4.82s)	Average loss=0.257527	Dev F1=0.00
[reRNN] Epoch 3 (6.50s)	Average loss=0.098538	Dev F1=0.00
[reRNN] Epoch 4 (8.18s)	Average loss=0.053479	Dev F1=0.00
[reRNN] Epoch 5 (9.92s)	Average loss=0.097188	Dev F1=0.00
[reRNN] Epoch 6 (11.64s)	Average loss=0.046231	Dev F1=0.00
[reRNN] Epoch 7 (13.48s)	Average loss=0.046012	Dev F1=0.00
[reRNN] Epoch 8 (15.31s)	Average loss=0.072844	Dev F1=0.00
[reRNN] Epoch 9 (17.15s)	Average loss=0.072371	Dev F1=0.00
[reRNN] Epoch 10 (19.01s)	Average loss=0.045975	Dev F1=0.00
[reRNN] Epoch 11 (20.85s)	Average loss=0.045975	Dev F1=0.00
[reRNN] Epoch 12 (22.72s)	Average loss=0.045779	Dev F1=0.00
[reRNN] Epoch 13 (24.53s)	Average loss=0.045566	Dev F1=0.00
[reRNN] Epoch 14 (26.42s)	Average loss=0.072123	Dev F1=0.00
[reRNN] Epoch 15 (28.24s)	Average loss=0.0446

In [29]:
from snorkel.annotations import load_gold_labels

In [30]:
L_gold_test = load_gold_labels(pipe.session, annotator_name='gold', split=2)

In [31]:
L_gold_test

<203x1 sparse matrix of type '<type 'numpy.int64'>'
	with 203 stored elements in Compressed Sparse Row format>

In [35]:
sum(L_gold_test.todense() == 1)

matrix([[12]])